# Introduction

This notebook shows how to get a analyse an architecture proposal for an architecture use case using OpenAI API

# Load OpenAI API key

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

if api_key:
    print(f"API key loaded: {api_key[:5]}... (truncated)")
else:
    print("API key not loaded")

API key loaded: sk-pr... (truncated)


# Create OpenAI client

In [2]:
from openai import OpenAI

try:
    openai_client = OpenAI()
    print("OpenAI client created successfully.")
except Exception as e:
    print(f"Failed to create OpenAI client: {e}")

OpenAI client created successfully.


# Upload pdf documents to OpenAI API Assistent

In [3]:
case = openai_client.files.create(
    file=open("Tech_Screening_Cloud_Architect.pdf", "rb"),
    purpose="assistants"
)
print("Uploaded file ID:", case.id)

Uploaded file ID: file-63xZLJN3Rdc325r37TDtZv


In [4]:
answer = openai_client.files.create(
    file=open("mixtapes.pdf", "rb"),
    purpose="assistants"
)
print("Uploaded file ID:", answer.id)

Uploaded file ID: file-AgxiidTtKZoMe3LqZv8ybk


In [15]:
# Check the file you have uploaded
files = openai_client.files.list()
for f in files.data:
    print(f"{f.id} | {f.filename} | {f.purpose} | {f.status}")

# Create API assistant

In [6]:
assistant = openai_client.beta.assistants.create(
    name="Document Analyzer",
    instructions="You analyse 2 documents, the case document presents a case and the answer document provides a response. You will analyse the case document. You will analyse the answer document. You will analyse how well the answer document is an answer on the case document and will descibe what aspects of the case are answered and what aspects are not answered and the output of this analysis is in markdown format. Also you will output some questions that can be asked (with the motivation of the question) to the writer of the answer document and output this in marksown format.",
    tools=[{"type": "file_search"}],
    model="gpt-4o"
)
print("Assistant ID:", assistant.id)

Assistant ID: asst_MzATDPG6Zi0Q45z0ZTIMsDey


# Create a thread

In [7]:
thread = openai_client.beta.threads.create()
print("Thread ID:", thread.id)

/var/folders/dr/zycbh73n35l83v_pny17zkmm0000gn/T/ipykernel_26426/3380736637.py:1: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  thread = openai_client.beta.threads.create()


Thread ID: thread_bNEjpn1mS9LwyiD0mbaes7kI


# Send a message with attached file

In [8]:
message = openai_client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=f"You analyse 2 documents, the case document with file_id = {answer.id} presents a case and the answer document with file_id = {case.id} provides a response. You will analyse the case document with file_id = {answer.id}. You will analyse the answer document with file_id = {case.id}. You will analyse how well the answer document with file_id = {case.id} is an answer on the case document with file_id = {answer.id} and will descibe what aspects of the case are answered and what aspects are not answered and the output of this analysis is in markdown format. Also you will output some questions that can be asked (with the motivation of the question) to the writer of the answer document with file_id = {answer.id}and output this in markdown format.",
    attachments=[
                {
                    "file_id": answer.id,
                    "tools": [{"type": "file_search"}]
                },
                {
                    "file_id": case.id,
                    "tools": [{"type": "file_search"}]
                }
            ]
)
print("Message ID:", message.id)

/var/folders/dr/zycbh73n35l83v_pny17zkmm0000gn/T/ipykernel_26426/4039859897.py:1: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  message = openai_client.beta.threads.messages.create(


Message ID: msg_kRVnHYLgP7feyaeoZ8laofIT


# Start a run and attach the pdf document

In [9]:
run = openai_client.beta.threads.runs.create(
            thread_id=thread.id,
            assistant_id=assistant.id,
        )

/var/folders/dr/zycbh73n35l83v_pny17zkmm0000gn/T/ipykernel_26426/3179579540.py:1: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run = openai_client.beta.threads.runs.create(


# Poll for result

In [10]:
import time
while True:
    run_status = openai_client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
    if run_status.status in ["completed", "failed"]:
        break
    time.sleep(1)

/var/folders/dr/zycbh73n35l83v_pny17zkmm0000gn/T/ipykernel_26426/295530023.py:3: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run_status = openai_client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)


# Read the assistant’s response

In [11]:
from IPython.display import Markdown, display

def display_markdown(text):
    display(Markdown(text))

In [12]:
messages = openai_client.beta.threads.messages.list(thread_id=thread.id)
for msg in reversed(messages.data):
    display_markdown(msg.content[0].text.value)

/var/folders/dr/zycbh73n35l83v_pny17zkmm0000gn/T/ipykernel_26426/2528440396.py:1: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  messages = openai_client.beta.threads.messages.list(thread_id=thread.id)


You analyse 2 documents, the case document with file_id = file-AgxiidTtKZoMe3LqZv8ybk presents a case and the answer document with file_id = file-63xZLJN3Rdc325r37TDtZv provides a response. You will analyse the case document with file_id = file-AgxiidTtKZoMe3LqZv8ybk. You will analyse the answer document with file_id = file-63xZLJN3Rdc325r37TDtZv. You will analyse how well the answer document with file_id = file-63xZLJN3Rdc325r37TDtZv is an answer on the case document with file_id = file-AgxiidTtKZoMe3LqZv8ybk and will descibe what aspects of the case are answered and what aspects are not answered and the output of this analysis is in markdown format. Also you will output some questions that can be asked (with the motivation of the question) to the writer of the answer document with file_id = file-AgxiidTtKZoMe3LqZv8ybkand output this in markdown format.

## Analysis of Documents

### Case Document: Mixtapes Case Study (file-AgxiidTtKZoMe3LqZv8ybk)

The case document outlines the strategic plan for migrating the Mixtapes platform to a scalable cloud infrastructure using Microsoft Azure. It provides a detailed comparison between the current AS-IS infrastructure hosted locally in Amsterdam and the proposed TO-BE architecture on Azure. Key highlights include:

- Reduced annual costs from €15,000 to €11,200 by migrating to Azure【4:0†mixtapes.pdf】.
- Improved global reach with Azure's multi-region deployments, enhancing user experience with lower latency【4:0†mixtapes.pdf】.
- Implementation of advanced features like Azure Kubernetes Service for scalability, Azure Blob Storage for cost-efficient data management, and Azure CDN for fast global delivery【4:6†mixtapes.pdf】【4:15†mixtapes.pdf】.
- Risk mitigation strategies such as data backups, geo-replication, and staged migrations to safeguard against data loss and ensure continuity【4:11†mixtapes.pdf】.

### Answer Document: Tech Screening Cloud Architect (file-63xZLJN3Rdc325r37TDtZv)

The answer document presents a scenario-based challenge for a Cloud Architect, where the focus is on proposing a cloud solution for the Mixtapes platform to support its international expansion. Important aspects covered include:

- Encouragement for open-ended solutions regarding what might be changed in the backend while ensuring low costs and improved manageability【4:12†Tech_Screening_Cloud_Architect.pdf】.
- Explanation of the current state of the Mixtapes platform and the opportunity to enhance it by exploiting cloud technologies for better scalability and performance【4:14†Tech_Screening_Cloud_Architect.pdf】.
- Emphasis on engaging non-technical stakeholders like Gert-Jan by simplifying complex technical concepts and focusing on business outcomes such as cost management and system reliability【4:5†Tech_Screening_Cloud_Architect.pdf】

### How Well the Answer Document Responds to the Case Document

The answer document does align broadly with the case document in terms of identifying the primary objectives such as cost-effectiveness, scalability, and system performance. It effectively centers on how a Cloud Architect might approach the problem, considering budget constraints and international expansion needs【4:4†Tech_Screening_Cloud_Architect.pdf】【4:5†Tech_Screening_Cloud_Architect.pdf】.

**Aspects Addressed:**

- **Scalability and Performance:** Both documents emphasize the need for a robust, scalable architecture that supports international growth【4:13†Tech_Screening_Cloud_Architect.pdf】.
- **Cost Efficiency:** The focus on maintaining cost-effective operations aligns with the financial benefits outlined in the case document【4:7†Tech_Screening_Cloud_Architect.pdf】.
- **Technical Improvements:** Features like redundant data backups and automated management are implicitly suggested, though not detailed explicitly in the answer document.

**Aspects Not Thoroughly Addressed:**

- **Specific Technical Solutions:** While the case document provides a clear roadmap and technology stack (e.g., Azure services), the answer document encourages creativity but does not specify particular technologies.
- **Detailed Migration Plan:** The technical specifics of migration, risk management, and operational management as detailed in the case are not explicitly covered in the answer【4:8†mixtapes.pdf】【4:9†mixtapes.pdf】.

## Questions for the Writer of the Answer Document

1. **Could you elaborate on the specific Azure services you would propose to use, and why, for each component (e.g., storage, compute, CDN)?**
   - *Motivation: To align with the cloud strategy detailed in the case document, providing clarity on how you envision the architecture leveraging Azure's capabilities.*

2. **What are the key milestones and timeline you foresee for implementing the migration to a cloud-based platform?**
   - *Motivation: To ensure the proposed plan is actionable and measurable with respect to the detailed phases in the case document.*

3. **How would you address potential data security and compliance issues during and after the migration process?**
   - *Motivation: Addressing concerns about data integrity and security which are crucial for stakeholder confidence, as detailed in the case document.*

4. **What specific steps would you recommend for risk mitigation and ensuring uninterrupted service during migration?**
   - *Motivation: To compare the risk management strategies with those laid out in the migration proposal of the case document.*

# Delete attached files

In [13]:
files = openai_client.files.list()
for f in files.data:
    openai_client.files.delete(file_id=f.id)

In [14]:
# Check if all files have been deleted

files = openai_client.files.list()
if not files.data:
    print("All files have been deleted successfully.")
else:
    print("Some files were not deleted:")
    for f in files.data:
        print(f"{f.id} | {f.filename} | {f.purpose} | {f.status}")

All files have been deleted successfully.
